## Rotor: Efisiensi vs Flow Coeff & Work Coeff


##### Imports


In [3]:
# IMPORTS
import numpy as np
import CoolProp
from CoolProp.CoolProp import PropsSI as Props
from CoolProp.CoolProp import PhaseSI as Phase
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

##### INPUTS: PARAMETERS & INLET OUTLET STATES

In [27]:
#INPUT TURBINE INLET & OUTLET FLUID STATE
def whichcycle (k):
    global T_1,T_5,P_1,P_5,mflow,fluid #Total Temperature (K),Total Pressure(P),'Nama fluida'
    if k == 1:
        T_1 = 360   
        T_5 = 335.4     #This value is obtained from Refprop. Will be replaced by value from CoolProp
        P_1 = 0.9   *1e6
        P_5 = 0.3   *1e6
        fluid = 'R245fa'
        mflow = 1.3
    if k == 2:
        T_1 = 400
        T_5 = 343.55
        P_1 = 1.22
        P_5 = 0.15079
        fluid = 'ISOPENTANE' 
        mflow = 0.5
    if k == 3:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5
    if k == 4:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5

In [50]:
    global k5h4,kb4b,rath5,rats5,rpm,Zratio,NR
    Rrh5r4  = 0.3   # ratio antara rh5 dengan r4 => 0<(rh5/r4) <1
    Rb5b4   = 1.2      # ratio antara b5 dengan b4 => b5/b4; preferrably >1
    # rath5   = 0.4
    # rats5   = 0.75
    rpm     = 20000
    Zratio  = 0.5
    NR      = 8


##### Define Compute Function

INPUT 10 x flow coeff dan 10x work coeff, serta siklus nomor berapa

In [56]:
def Compute(tenflow_coeff,tenwork_coeff,k): #INPUT 10 X flow atau work coeff

    global  T_1,T_5,P_1,P_5,p04s,p05ss,T04s,T05ss
    global  TT4,T4s,p4,p4s,T5,p5
    global  NR,r4,Alpha4,b4,Ct4,rho4,Beta5,beta5didconverge,Beta4opt
    global cu5,cu5num,cu4,errorbeta5,beta,U5,U4,r5,DeltaH,C4,C5,W4,W5,C5_0,C5i,C5ii,b5,b4
    global Cm4,Ct4,U4,Alpha4,Beta4,Cm5,Ct5,U5,Alpha5,Beta5,beta5didconverge,Zr
    global rho04s,rho4s,a4,C4,h5ss,h05ss,h04s,h4s,a01,C5didconverge,kC5,errorC5
    global TotalLoss,LossInc,LossPass,LossTip,LossWind,LossTE,LossExit,rho4m
    global s04s,T04,T04s,p04,p04s,s04,etaspred,rho4sm,rho4,rho04,a5ss,rho5ss,p5ss,k1C5,k2C5
    flow_coeff=tenflow_coeff/10
    work_coeff=tenwork_coeff/10
    whichcycle (k)       # The cycle to be computed is the k
    
    Cp4 = Props('C','T',T_1,'P',P_1,fluid)
    Cv4 = Props('O','T',T_1,'P',P_1,fluid)
    gamma = Cp4/Cv4
    Rx = 8.31446261815324   #J/K.mol

    #General Properties inlet outlet turbin (Total)
    H_1     = Props('H','T',T_1,'P',P_1,fluid)     #J/kg
    s01     = Props('S','T',T_1,'P',P_1,fluid)     #J/kg.K 
    T_5     = Props('T','P',P_5,'S',s01,fluid)  # =>asumsi nozzle isenthalpy DAN Isentropic
    H_5     = Props('H','T',T_5,'P',P_5,fluid)  # meski pada kenyataannya isenthalpic nozzle tidak isentropic
    DeltaH  = H_1-H_5            #Ideal === Isentropic Total Enthalpy change 

    C0s     = np.sqrt(2*DeltaH)         #Spouting Velocity

    #Segitiga Kecepatan Inlet, m/s, radians
    U4      = np.sqrt(DeltaH/work_coeff)
    Cm4     = U4*flow_coeff
    Ct4     = DeltaH/U4                 # => DeltaH = U4*Ct4-U5*Ct5 ; Alpha5=0 => Ct5=0
    C4      = np.sqrt(Cm4**2+Ct4**2)
    Alpha4  = np.arctan(Ct4/Cm4)
    W4      = np.sqrt(Cm4**2+(U4-Ct4)**2)
    Beta4   = np.arctan((U4-Ct4)/Cm4)

    #Perhitungan Properties ideal lain (Total)
    p01     = P_1           #inlet volute [1], Total
    T01     = T_1
    h01     = H_1
    p1      = p01
    T1      = T_1
    h01     = H_1
    rho1   = Props('D','P',p1,'T',T1,fluid)
    
    p05ss   = P_5
    T05ss   = T_5
    h05ss   = H_5
    s05ss   = Props('S','H',h05ss,'P',p05ss,fluid)
    rho05ss = Props('D','P',p05ss,'T',T05ss,fluid)

    Q5      = mflow/rho05ss
    ns      = np.radians(rpm*6)*np.sqrt(Q5)/DeltaH**0.75
    etaspred    = 0.81-1.07*(ns-0.55)**2-0.5*(ns-0.55)**3       #predicted total-to-static efficiency

    
    h02s    = H_1           #inlet nozzle [2], Total
    s02s    = s01            #ideal volute === approx. as isentropic
    p02s    = p01
    T02s    = T01
    h03s    = h02s           #outlet nozzle [3], Total
    s03s    = s02s            #ideal nozzle === approx. as isentropic (in Total)
    p03s    = p02s
    T03s    = T02s
    h04s    = h03s           #inlet rotor [4], Total
    s04s    = s03s           #outlet nozzle === inlet rotor
    p04s    = p03s
    T04s    = T03s
    h04     = h04s          # Nozzle isenthalpic but not isentropic
    p04     = p01-rho1*DeltaH*(1-etaspred)/4
    T04     = Props('T','P',p04,'H',h04s,fluid)
    s04     = Props('S','P',p04,'T',T04,fluid)
    
    h05ss   = H_5
    p05ss   = P_5
    T05ss   = T_5
    s05ss   = Props('S','H',h05ss,'P',p05ss,fluid)

    #Perhitungan Properties ideal lain (Static)
    h4s     = h04s-1/2*C4**2
    p4s     = Props('P','H',h4s,'S',s04s,fluid)
    p4     = Props('P','H',h4s,'S',s04,fluid)
    T4s     = Props('T','H',h4s,'S',s04s,fluid)
    rho04s  = Props('D','P',p04s,'T',T04s,fluid)
    rho4s   = Props('D','P',p4s,'T',T4s,fluid)
    rho4sm   = 2*(p04s-p4s)/C4**2
    h4      = h04-1/2*C4**2
    p4      = Props('P','H',h4,'S',s04,fluid)
    T4      = Props('T','H',h4,'S',s04,fluid)
    rho04   = Props('D','P',p04,'T',T04,fluid)
    rho4    = Props('D','P',p4,'H',h04,fluid)
    rho4m   = 2*(p04-p4)/C4**2
    a01     = Props('A','P',p01,'T',T01,fluid)
    a4s     = Props('A','P',p4s,'T',T4s,fluid)
    a4      = Props('A','P',p4,'T',T4,fluid)
    Ma4s    = C4/a4s
    Ma4     = C4/a4



    #Perhitungan Geometri 
    r4  = U4/np.radians(rpm*6)
    b4  = mflow/(2*np.pi*r4*rho4s*Cm4)
    D4  = r4*2
    Zr  = Zratio*r4
    rh5 = Rrh5r4*r4
    b5  = Rb5b4*b4  
    rs5 = rh5+b5
    r5  = (rs5+rh5)/2

    Re4s    = rho4s*C4*b4/Props('V','P',p4s,'T',T4s,fluid)
    Re4     = rho4*C4*b4/Props('V','P',p4,'T',T4,fluid)

    #Segitiga Kecepatan Outlet
    U5      = r5*np.radians(rpm*6)
    
    rho5ss_0 = rho05ss
    C5ii   = C5_0
    rho5ssii= rho5ss_0
    C5didconverge1 = False
    C5didconverge2 = False
    k1C5    = 0
    k2C5    =0
    while C5didconverge1 == False:
        k1C5     = k1C5+1             # => iteration number
        C5i      = C5ii
        rho5ssi = rho5ssii
        C5ii    = mflow/(rho5ssi*2*np.pi*r5*b5)
        h5ss    = h05ss-1/2*C5i**2
        rho5ssii  = Props('D','H',h5ss,'S',s05ss,fluid)
        errorC5= mflow/(rho5ssii*C5ii*2*np.pi*b5*r5)-1
        if errorC5 <= 10**-10:
            C5didconverge1 = True
            C5didconverge2 = True
            C5  = C5ii
            rho5ss=rho5ssii
            break
        if (rho5ssi*C5i-rho5ssii*C5ii)*(C5i-C5ii)>0:
            C5      = C5ii
            rho5ss  = rho5ssii
            break
    while C5didconverge2 == False:
        k2C5    = k2C5 +1
        C5      = mflow/(rho5ss*2*np.pi*r5*b5)
        h5ss    = h05ss-1/2*C5**2
        rho5ss  = Props('D','H',h5ss,'S',s05ss,fluid)
        h5ss    = h05ss-1/2*C5i**2
        errorC5= mflow/(rho5ss*C5*2*np.pi*b5*r5)-1
        if errorC5 <= 10**-10:
            break
    Alpha5  = 0
    Ct5     = C5*np.sin(Alpha5)
    Cm5     = C5*np.cos(Alpha5)
    Beta5   = np.arctan((U5-Ct5)/Cm5)
    W5      = Cm5/np.cos(Beta5)

    p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
    a5ss    = Props('A','H',h5ss,'P',p5ss,fluid)
    Ma5ss   = C5/a5ss


    # \\\\\\\ <<---------<<----||----->>------------>> ////////
    ## Losses Coefficient ##

    #Rotor Incidence Losses
    Beta4opt = np.arctan((-1.98/NR)/NR/(1-1.98/NR)*np.tan(Alpha4))
    LossInc = 0.5*(W4**2)*(np.sin(np.abs(Beta4-Beta4opt)))**2  #m2/s2

    #Rotor Passage Losses
    LH = np.pi/4*((Zr-b4/2)+(r4-rh5-b5/2))                                                              #m
    DH = 0.5*((4*np.pi*r4*b4/(2*np.pi*r4+Zr*rh5))+((2*np.pi*(rs5**2-rh5**2)/(np.pi*(rs5-rh5))+Zr*b5)))  #m
    Y5 = np.arctan(0.5*(np.tan(Beta4)+np.tan(Beta5)))
    C = Zr/np.cos(Y5)
    if (r4-rs5)/b5>=0.2:
        KpCETI = 0.11
    else:
        KpCETI = 0.22
    LossPass = KpCETI*(LH/DH+0.68*((1-(r5/r4)**2)*np.cos(Beta5)/(b5/C))*((W4**2+W5**2)/2))
    
    #Rotor Clearance Losses
    Ca = (1-(rs5/r4))/(Cm4*b4)
    Cr = (rs5/r4)*((Zr-b4)/(Cm5*r5*b5))
    Ka = 0.4
    Kr = 0.75
    Kar = -0.3
    Ea = 0.0003
    Er = 0.0003
    LossTip = (U4**3*NR/(8*np.pi))*(Ka*Ea*Ca+Kr*Er*Cr+Kar*np.sqrt(Ea*Er*Ca*Cr))

    #Windage Losses
    Eb = 0.0003
    Kf = 3.7*(Eb/r4)**0.1/Re4s**0.5
    LossWind = Kf*((rho4s+rho5ss)/2)*U4**3*r4**2/(2*mflow*W5**2)

    #Trailing Edge Losses
    tb4 = 0.04*r4
    tb5 = 0.02*r4
    LossTE = rho5ss*W5**2/2*(NR*tb5/(np.pi*(rh5+rs5)*np.cos(Beta5)))**2
    
    #Exit Losses
    LossExit = 0.5*C5**2    # => mungkin untuk ubah dari total jadi static. abaikan dulu

    # => Sum Enthalpy Losses
    TotalLoss = LossInc + LossPass + LossTip + LossWind + LossTE
    # \\\\\\\ <<---------<<----||----->>------------>> ////////


    #Perhitungan Properties considering losses
    h05 = h05ss-TotalLoss#bentar masih salah lossnya       #nozzle masih diasumsikan isentropic dan isenthalpic
    h5  = h05-1/2*C5**2     
    p05 = p05ss
    T05 = Props('T','H',h05,'P',p05,fluid)
    p5  = p5ss
    T5  = Props('T','H',h5,'P',p5,fluid)




##### Run tests

In [61]:
Compute(1.6,10.4,1)
print(C5)
print(p04,p04s,p4,p4s,T4,T4s)
print(r4,b4,r5,b5)
print(DeltaH,etaspred,U4,Ct4)
print(np.degrees(Beta5),np.degrees(Beta4opt),np.degrees(Beta4),np.degrees(Alpha4))
print(TotalLoss)
print(LossInc,LossPass,LossTip,LossWind,LossTE)
print(rho4,rho4s,etaspred)

89.98705709531237
841886.569552589 900000.0 469729.3211872201 499606.11506475834 341.1569334935949 341.79184446806937
0.06693862290530606 0.005159902949740917 0.023177528641436365 0.0061918835396891
20441.092982827453 0.7708252950367916 140.19592397381797 145.8037609327707
28.344338962975183 -14.96177235904736 -14.036243467926509 81.2538377374448
2878.1262421507795
0.06974396015572591 1738.794939244239 554.4880883514858 0.017163651976465695 584.7563069429228
23.840201384758224 26.704829358734877 0.7708252950367916


In [67]:
Compute(1,9,1)
print(s04s)
print(s04)
print(p04)
print(p04s)
print(etaspred)
print("rho04s=",rho04s)
print("rho4s=",rho4s)
print("rho4sm=",rho4sm)
print("rho04=",rho04)
print("rho4=",rho4)
print("rho4m=",rho4m)
print("rho5ss=",rho5ss)
print("rho05=",Props('D','T',T_5,'P',P_5,fluid))
print(Beta4,Beta4opt)
print(C5/a5ss)
print(U4*Ct4,DeltaH)

1793.4836287876885
1796.5188773395316
847733.5500283497
900000.0
0.7200034439549657
rho04s= 49.621022517968555
rho4s= 30.53211939543192
rho4sm= 39.17280489841818
rho04= 46.2386200752728
rho4= 27.65647780871778
rho4m= 36.6559349881459
rho5ss= 21.125170474790245
rho05= 21.326260040271254
0.0 -0.39006386069841936
0.13762829324080908
16719.45381859039 15047.508436731354


In [ ]:

print(cu(ruc(0,np.radians(70)),rcw(0,np.radians(70))))
Compute(0.2,1.1,2)
print("DeltaH=",DeltaH)
print((Ct4*U4-Ct5*U5))
print(np.abs(DeltaH-((Ct4*U4-Ct5*U5)))/DeltaH*100,"%")
print(r4)
print(r5)
print(U4)
print(U5)
print(cu4)
print(fluid)
print("cu5=",cu5)
print("cu5num=",cu5num)
print("converge?=",beta5didconverge)
print("Error Beta5=",errorbeta5)
print("Beta5=",np.degrees(Beta5))

print(TotalLoss)
print(LossInc,LossPass,LossTip,LossWind,LossTE)

In [ ]:
def testing1
    Compute(0.1,0.9,1)
    # print(h04s)
    # print(h4s)
    # print(h05ss)
    # print(h5ss)
    # print("w(by enthalpy)=",DeltaH)
    # print("w(by enthalpy)=",h04s-h05ss)
    # print("w(by vel)=",-1/2*((U5**2-U4**2)+(C5**2-C4**2)-(W5**2-W4**2)))
    # print("W(by velCtU)=",Ct4*U4-Ct5*U5)
    # print("W(by velCtU)=",Ct4*U4-C5*np.sin(Alpha5)*U5)
    print(C4/a01)
    print(C4/a4)
    print("Alpha4=",np.degrees(Alpha4))
    print("Beta4=",np.degrees(Beta4))
    print("Cm4=",Cm4)
    print("C4=",Ct4)
    print("Ct4=",C4)
    print("U4=",U4)
    print("rho4s=",rho4s)
    print("rho4s2=",rho4s2)
    print("Alpha5=",np.degrees(Alpha5))
    #print("beta5converge?=",beta5didconverge)
    print("C5 converged? =",C5didconverge)
    print("iterated for",kC5,"times")
    print("error C5=",errorC5)
    print("Beta5=",np.degrees(Beta5))
    print("Cm5=",Cm5)
    print("C5=",C5)
    print("Ct5=",Ct5)
    print("U5=",U5)
    print("r4=",r4)
    print("r5=",r5)
    print("b5=",b5,"b4=",b4)
    print("Zr=",Zr)
    print(TotalLoss)
    print(LossInc,LossPass,LossTip,LossWind,LossTE)

In [ ]:
print(DeltaH,h04s,h05ss,C5i,kC5)

##### Archives, Scrapped out parts

In [4]:
def rcw (Alpha,Beta):   #input in radians
    return (np.cos(Alpha)/np.cos(Beta))
def ruc (Alpha,Beta):
    return (1/(np.sin(Alpha)+np.tan(Beta)*np.cos(Alpha)))
def cu (ruc,rcw):
    return (1-(ruc*rcw)**2+ruc**2)

In [ ]:
    #rho04s    = Props('D','T',T_1,'P',P_1,fluid)  #  rho4~= rho1?
    #rho05s    = Props('D','T',T_5,'P',P_5,fluid)  #kg/m3
    #rho dihitung pada static

    # Beta4   = np.radians(30)                         # =>INPUT Control Variable(rads)
    # U4      = np.sqrt(DeltaH/work_coeff)
    # Cm4     = U4*flow_coeff
    # W4      = Cm4/np.cos(Beta4)
    # Ct4     = U4-W4*np.sin(Beta4) 
    # Alpha4  = np.arctan(Ct4/Cm4)  #in radians           
    # C4      = Cm4/np.cos(Alpha4)
    # 
        #Segitiga Kecepatan Outlet
    # Alpha5  = np.radians(1)    # =>INPUT initial Control Variable(rads)
    #U5      = r5*np.radians(rpm*6)
    #cu4     = cu(ruc(Alpha4,Beta4),rcw(Alpha4,Beta4))
    #cu5     = (U4**2*cu4-2*DeltaH)/U5**2
    #     #Mencari nilai Beta5 dengan galat cu < 0.5%
    # beta5didconverge = False    #initial state
    # Beta5 = np.degrees(0.1)
    # cu5num = cu(ruc(Alpha5,Beta5),rcw(Alpha5,Beta5))
    # errorbeta5  = np.abs((cu5-cu5num)/cu5)
    # while errorbeta5>0.005:
    #     Beta5 = Beta5+np.radians(0.01)
    #     cu5num = cu(ruc(Alpha5,Beta5),rcw(Alpha5,Beta5))
    #     errorbeta5  = np.abs((cu5-cu5num)/cu5)
    #     if errorbeta5<=0.01:
    #         beta5didconverge = True
    #     if Beta5 >= np.radians(180):
    #         break
    #==> dicoba dulu pakai uct
    #Ct5     = (U4*Ct4-DeltaH)/U5
    #Cm5 = U4/(np.tan(Alpha5)+np.tan(Beta5))
    #W5  = Cm5/np.cos(Beta5)
    #C5  = Cm5/np.cos(Alpha5)
    #Ct5 = Cm5*np.tan(Alpha5)
    # Re4     = rho4s*C4*b4/Props('V','P',p04s,'T',T04s,fluid)
    # #Properties ideal static di 5
    # h5ss    = h05ss-1/2*C5**2
    # p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
    # rho5ss  = 2*(p05ss-p5ss)/C5**2
    #Segitiga Kecepatan outlet versi 2
    #Alpha5  = np.radians(20)     #  ==> INPUT Control variable
    #--------------------------====================================> V dihold dulu karena hasilnya aneh
    # C5didconverge = False 
    # C5_0    = (U4*Ct4-DeltaH)/(np.sin(Alpha5)*U5) #i=0
    # C5ii    = C5_0  #trick untuk menghemat baris
    # #errorC5 = np.abs((C5i-C5ii)/C5i)
    # kC5     = 0
    # while C5didconverge == False:
    #     kC5     = kC5 +1
    #     C5i     = C5ii
    #     h5ss    = h05ss-1/2*C5i**2
    #     #p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
    #     rho5ss  = Props('D','H',h5ss,'S',s05ss,fluid)
    #     #rho5ss  = 2*(p05ss-p5ss)/C5i**2                => scrap out
    #     C5ii    = mflow/(2*np.pi*b5*r5*rho5ss)
    #     errorC5 = np.abs((C5i-C5ii)/C5i)
    #     if errorC5 <= 0.000005:
    #         C5didconverge = True
    #         C5  = C5ii
    # U5      = (U4*Ct4-DeltaH)/(np.sin(Alpha5)*C5)
    # Cm5     = C5*np.cos(Alpha5)
    # Ct5     = C5*np.sin(Alpha5)
    # Beta5   = np.arctan((U5-Ct5)/Cm5)
    # W5      = Cm5/np.cos(Beta5)
    # r5      = U5/np.radians(rpm*6)
    # h5ss    = h05ss-1/2*C5**2
    # p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
    # T5ss    = Props('T','H',h5ss,'S',s05ss,fluid)
    # #rho5ss  = 2*(p05ss-p5ss)/C5**2
    #-------------------------------------
    
    #C5_0    = mflow/(rho05ss*2*np.pi*r5*b5)     # => C5 initial guess from mass balance
    #Ct5     = (U4*Ct4-DeltaH)/U5
    C5_0     = 0